In [79]:
##########################################################################
##### Template code and instructions to scrape Rate My Professor Comments#
##########################################################################
### Step 1: install packages - 
##### packages are necessary to install and load, given that they have the built in functions necessary to run complex tasks. 
## They effectively act as one of the most crucial time saving activities that would otherwise lead to overly long and 
## duplicative scripts. 

## install pkgs 
import sys
!{sys.executable} -m pip install numpy
# !{sys.executable} -m pip install requests #; this code here can be used to install packages on anaconda/jupyter notebook 
### I believe the below should be installed by default 
import requests # web scraping 
from bs4 import BeautifulSoup # for web scraping 
import itertools # for efficient operation of loops 
import pandas as pd # necessary for reading in, creating, and manipulating data frames 
import csv ## for importing/exporting csvs 
import glob ## for finding files in path
import re


In [80]:
### Step 2: Read in the csv with the data of interest 
## note: This will be the section that you update manually the most. The rest should be automated 

faculty_csv = pd.read_csv('faculty characteristics web scraping_UNC.csv')
urls = faculty_csv['link_rmp']


## url to use ; will be the profs page, with the numeric code being what changes; should be changed manually 

#url = ['https://www.ratemyprofessors.com/professor/1190096','https://www.ratemyprofessors.com/professor/2070585']

## college 
college = faculty_csv['college']

##prof last name 
prof_lastname = faculty_csv['prof_lastname']

# prof first name 
prof_firstname = faculty_csv['prof_firstname']

## note: you will want to manually seach the rate my professor website for the professor of interest. From there, you will 
# be able to grab the url for the professor of interest, which will be the last numeric digits of the url that will vary. 
#url
faculty_csv

,college,prof_firstname,prof_lastname,program,male,non_white,link_rmp,notes
0,UNIVERSITY OF NORTH CAROLINA,DALE,WHITTINGTON,ENVIRONMENTAL SCIENCES & ENGINEERING,1,0,https://www.ratemyprofessors.com/professor/100...,NaN
1,UNIVERSITY OF NORTH CAROLINA,COURTNEY,WOODS,ENVIRONMENTAL SCIENCES & ENGINEERING,0,1,https://www.ratemyprofessors.com/professor/225...,NaN
2,UNIVERSITY OF NORTH CAROLINA,KARI,NORTH,EPIDEMIOLOGY,0,0,https://www.ratemyprofessors.com/professor/212...,NaN
3,UNIVERSITY OF NORTH CAROLINA,SUZANNE,MAMAN,HEALTH BEHAVIOR,0,0,https://www.ratemyprofessors.com/professor/286...,Ratings are for Health Science
4,UNIVERSITY OF NORTH CAROLINA,BENJAMIN,MEIER,HEALTH POLICY & MANAGEMENT,1,0,https://www.ratemyprofessors.com/professor/143...,Ratings are for Public Policy
...,...,...,...,...,...,...,...,...
97,UNIVERSITY OF NORTH CAROLINA,TED,MOUW,SOCIOLOGY,1,0,https://www.ratemyprofessors.com/professor/379594,NaN
98,UNIVERSITY OF NORTH CAROLINA,LISA,PEARCE,SOCIOLOGY,0,0,https://www.ratemyprofessors.com/professor/221...,NaN
99,UNIVERSITY OF NORTH CAROLINA,ALEXANDREA,RAVENELLE,SOCIOLOGY,0,0,https://www.ratemyprofessors.com/professor/259...,NaN
100,UNIVERSITY OF NORTH CAROLINA,JESSICA,SU,SOCIOLOGY,0,0,https://www.ratemyprofessors.com/professor/252...,NaN


In [81]:
### Step 3: run nested loop with these data

for j, element in enumerate(urls):
    temp_page = requests.get(element) ## grabbing the page from the url, which is stored in the jth row of the url list
    soup = BeautifulSoup(temp_page.text, "html.parser") # parse the html for the jth pulled page 
    prof_comments = soup.findAll("div", {"class": "Comments__StyledComments-dzzyvm-0 gRjWel" }) # grab the comments 
    prof_score = soup.findAll("div", {"class": "CardNumRating__CardNumRatingNumber-sc-17t4b9u-2" }) # grab scores 
    prof_class = soup.findAll("div", {"class": "RatingHeader__StyledClass-sc-1dlkqw1-2 gxDIt" }) # just take the odds
    comment_thumbs = soup.findAll("div", {"class": "Thumbs__HelpTotal-sc-19shlav-1" }) # thumbs up and down
    ## split up into difficulty, quality and class num 
    prof_dif = prof_score[1::2]
    prof_qual = prof_score[0::2]
    prof_class = prof_class[1::2]
    thumbs_down = comment_thumbs[1::2]
    #rint(thumbs_down)
    thumbs_up = comment_thumbs[0::2]
    #rint(thumbs_up)
    ### now a new loop to store the scraped data from the jth row into the new temp df 
    d1 = []
    for i, element in enumerate(prof_qual):
        if len(thumbs_up) == 0:
            thumbs_up2 = 0
        else:
            thumbs_up2 = thumbs_up[i].get_text()
        if len(thumbs_down) == 0:
            thumbs_down2 = 0
        else:
            thumbs_down2= thumbs_down[i].get_text()
        d1.append({
            'row': i+1,
                    'quality_of_class': prof_qual[i].get_text(),
                    "difficulty_of_class": prof_dif[i].get_text(),
                    "thumbs_up": thumbs_up2,
                    "thumbs_down": thumbs_down2,
                    #class_code": prof_class[i].get_text(),
            #"college": college[j], #jth lvl, not ith level, variable 
            #"prof_firstname": prof_firstname[j],            
            #"prof_lastname": prof_lastname[j], # same logic as above, now for the prof's last name         
                'comment': prof_comments[i].get_text() #ith level again 
        }
        )
    df = pd.DataFrame(d1)
    ## add j level vars here
    df['college'] = college[j]
    df['prof_firstname'] = prof_firstname[j]
    df['prof_lastname'] = prof_lastname[j]
    ### now save the file 
    save_name = "scraped_data/FINALUNCTHUMBS" +"_"+college[j] +"_"+prof_lastname[j]+"_"+prof_firstname[j]+".csv"
    df.to_csv(save_name, index=False)







In [82]:
#### compile all of the files in the directory ## 
scraped_data_files =  glob.glob("scraped_data/*")
final_thumbs_scraped_data_files_unc = [x for x in scraped_data_files if re.findall("FINALUNCTHUMBS",x)] 
# grab files that have "zest" in title
final_thumbs_scraped_data_files_unc


['scraped_data\\FINALUNCTHUMBS_UNIVERSITY OF NORTH CAROLINA_ABRAMOWITZ_JONATHAN.csv',
 'scraped_data\\FINALUNCTHUMBS_UNIVERSITY OF NORTH CAROLINA_ALDRICH_HOWARD.csv',
 'scraped_data\\FINALUNCTHUMBS_UNIVERSITY OF NORTH CAROLINA_ALGOE_SARA.csv',
 'scraped_data\\FINALUNCTHUMBS_UNIVERSITY OF NORTH CAROLINA_ANDERSON_ASHLEY.csv',
 'scraped_data\\FINALUNCTHUMBS_UNIVERSITY OF NORTH CAROLINA_ANDREWS_KENNETH.csv',
 'scraped_data\\FINALUNCTHUMBS_UNIVERSITY OF NORTH CAROLINA_ARNOLD_JENNIFER.csv',
 'scraped_data\\FINALUNCTHUMBS_UNIVERSITY OF NORTH CAROLINA_BALLARD-ROSA_CAMERON.csv',
 'scraped_data\\FINALUNCTHUMBS_UNIVERSITY OF NORTH CAROLINA_BAPAT_NAVIN.csv',
 'scraped_data\\FINALUNCTHUMBS_UNIVERSITY OF NORTH CAROLINA_BARDONE-CONE_ANNA.csv',
 'scraped_data\\FINALUNCTHUMBS_UNIVERSITY OF NORTH CAROLINA_BASSI_ANNA.csv',
 'scraped_data\\FINALUNCTHUMBS_UNIVERSITY OF NORTH CAROLINA_BAUCOM_DONALD.csv',
 'scraped_data\\FINALUNCTHUMBS_UNIVERSITY OF NORTH CAROLINA_BAUER_DANIEL.csv',
 'scraped_data\\FINALUNCT

In [84]:
final_thumbs_master_unc = pd.DataFrame()
  
for element in final_thumbs_scraped_data_files_unc:
    temp_df = pd.read_csv(element)
    final_thumbs_master_unc = final_thumbs_master_unc.append(temp_df)

    
           
        
    
    
    
    
    

C:\Users\Gabem\AppData\Local\Temp\ipykernel_22124\1121358816.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_thumbs_master_unc = final_thumbs_master_unc.append(temp_df)
C:\Users\Gabem\AppData\Local\Temp\ipykernel_22124\1121358816.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_thumbs_master_unc = final_thumbs_master_unc.append(temp_df)
C:\Users\Gabem\AppData\Local\Temp\ipykernel_22124\1121358816.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_thumbs_master_unc = final_thumbs_master_unc.append(temp_df)
C:\Users\Gabem\AppData\Local\Temp\ipykernel_22124\1121358816.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inste

In [ ]:
thumbs_master_unc

In [85]:
### now export 
final_thumbs_master_unc.to_csv("final_thumbs_master_unc_data_comments.csv", index=False)


In [10]:
#### Import the OSU data 
faculty_csv = pd.read_csv('faculty characteristics web scraping_OSU2.csv')

faculty_csv=faculty_csv.loc[faculty_csv['link_rmp'] != "DNE"]

### lets reindex 
#j_range = list(range(0,len(faculty_csv),1))
#print(j_range)

urls = faculty_csv['link_rmp']


## url to use ; will be the profs page, with the numeric code being what changes; should be changed manually 

#url = ['https://www.ratemyprofessors.com/professor/1190096','https://www.ratemyprofessors.com/professor/2070585']

## college 
college = faculty_csv['college']

##prof last name 
prof_lastname = faculty_csv['prof_lastname']

# prof first name 
prof_firstname = faculty_csv['prof_firstname']

## note: you will want to manually seach the rate my professor website for the professor of interest. From there, you will 
# be able to grab the url for the professor of interest, which will be the last numeric digits of the url that will vary. 
#url
faculty_csv




,college,prof_firstname,prof_lastname,program,male,non_white,link_rmp,NOTES
0,OHIO STATE UNIVERSITY,NAOMI,ADANIYA,PUBLIC HEALTH,0,1,https://www.ratemyprofessors.com/professor/207...,NaN
1,OHIO STATE UNIVERSITY,SARA,ANDERSON,PUBLIC HEALTH,0,0,https://www.ratemyprofessors.com/professor/272...,NaN
2,OHIO STATE UNIVERSITY,REBECCA,ANDRIDGE,PUBLIC HEALTH,0,1,https://www.ratemyprofessors.com/professor/240...,NaN
3,OHIO STATE UNIVERSITY,KELLIE,ARCHER,PUBLIC HEALTH,0,0,https://www.ratemyprofessors.com/professor/245...,NaN
4,OHIO STATE UNIVERSITY,GIA,BARBOZA-SALERNO,PUBLIC HEALTH,0,0,https://www.ratemyprofessors.com/professor/287...,NaN
...,...,...,...,...,...,...,...,...
225,OHIO STATE UNIVERSITY,JOHN,MUELLER,POLITICAL SCIENCE,1,0,https://www.ratemyprofessors.com/professor/175682,NaN
226,OHIO STATE UNIVERSITY,RANDALL,RIPLEY,POLITICAL SCIENCE,1,0,https://www.ratemyprofessors.com/professor/811378,NaN
227,OHIO STATE UNIVERSITY,GOLDIE,SHABAD,POLITICAL SCIENCE,0,0,https://www.ratemyprofessors.com/professor/93027,NaN
228,OHIO STATE UNIVERSITY,ELLIOT,SLOTNICK,POLITICAL SCIENCE,1,0,https://www.ratemyprofessors.com/professor/51211,NaN


In [11]:
len(urls)

230

In [43]:
### Step 3: run nested loop with these data

for j, element in enumerate(urls):
    temp_page = requests.get(element) ## grabbing the page from the url, which is stored in the jth row of the url list
    soup = BeautifulSoup(temp_page.text, "html.parser") # parse the html for the jth pulled page 
    prof_comments = soup.findAll("div", {"class": "Comments__StyledComments-dzzyvm-0 gRjWel" }) # grab the comments 
    prof_score = soup.findAll("div", {"class": "CardNumRating__CardNumRatingNumber-sc-17t4b9u-2" }) # grab scores 
    prof_class = soup.findAll("div", {"class": "RatingHeader__StyledClass-sc-1dlkqw1-2 gxDIt" }) # just take the odds
    comment_thumbs = soup.findAll("div", {"class": "Thumbs__HelpTotal-sc-19shlav-1" }) # thumbs up and down
    # <div id="thumbs-id" class="Thumbs__ButtonWrapper-sc-19shlav-0 jsOpVQ"><div id="thumbs_up" class="Thumbs__HelpTotal-sc-19shlav-1 gbMMhq"><img src="/static/media/thumbs-up-black.eddae738.svg" class="VoteThumb__StyledVoteThumb-p2gtch-0 feDtbj" data-tooltip="true" data-tip="Helpful" data-for="GLOBAL_TOOLTIP" alt="Thumbs up" currentitem="false"> <!-- -->1</div><div id="thumbs_down" class="Thumbs__HelpTotal-sc-19shlav-1 gbMMhq"><img src="/static/media/thumbs-down-black.bd601b36.svg" class="VoteThumb__StyledVoteThumb-p2gtch-0 feDtbj" data-tooltip="true" data-tip="Not helpful" data-for="GLOBAL_TOOLTIP" alt="Thumbs down" currentitem="false"> <!-- -->1</div></div>
    # <div id="thumbs_up" class="Thumbs__HelpTotal-sc-19shlav-1 gbMMhq"><img src="/static/media/thumbs-up-black.eddae738.svg" class="VoteThumb__StyledVoteThumb-p2gtch-0 feDtbj" data-tooltip="true" data-tip="Helpful" data-for="GLOBAL_TOOLTIP" alt="Thumbs up" currentitem="false"> <!-- -->0</div>
    ## split up into difficulty, quality, class num, thumbs up and thumbs down
    prof_dif = prof_score[1::2]
    prof_qual = prof_score[0::2]
    prof_class = prof_class[1::2]
    thumbs_down = comment_thumbs[1::2]
    #rint(thumbs_down)
    thumbs_up = comment_thumbs[0::2]
    #rint(thumbs_up)
    ### now a new loop to store the scraped data from the jth row into the new temp df 
    d1 = []        
    for i, element in enumerate(prof_qual):
        if len(thumbs_up) == 0:
            thumbs_up2 = 0
        else:
            thumbs_up2 = thumbs_up[i].get_text()
        if len(thumbs_down) == 0:
            thumbs_down2 = 0
        else:
            thumbs_down2= thumbs_down[i].get_text()
        d1.append({
            'row': i+1,
                    'quality_of_class': prof_qual[i].get_text(),
                    "difficulty_of_class": prof_dif[i].get_text(),
                    "thumbs_up": thumbs_up2,
                    "thumbs_down": thumbs_down2,
                    #class_code": prof_class[i].get_text(),
            #"college": college[j], #jth lvl, not ith level, variable 
            #"prof_firstname": prof_firstname[j],            
            #"prof_lastname": prof_lastname[j], # same logic as above, now for the prof's last name         
                'comment': prof_comments[i].get_text() #ith level again 
        }
        )

    df = pd.DataFrame(d1)
    ## add j level vars here
    df['college'] = college[j]
    df['prof_firstname'] = prof_firstname[j]
    df['prof_lastname'] = prof_lastname[j]
    
    ### now save the file 
    save_name = "scraped_data/FINALTHUMBS" +"_"+college[j] +"_"+prof_lastname[j]+"_"+prof_firstname[j]+".csv"

    df.to_csv(save_name, index=False)






In [ ]:
### loop for the missing profs 
for j, element in enumerate(urls):
    print(j)
    temp_page = requests.get(element) ## grabbing the page from the url, which is stored in the jth row of the url list
    soup = BeautifulSoup(temp_page.text, "html.parser") # parse the html for the jth pulled page 
    prof_comments = soup.findAll("div", {"class": "Comments__StyledComments-dzzyvm-0 gRjWel" }) # grab the comments 
    prof_score = soup.findAll("div", {"class": "CardNumRating__CardNumRatingNumber-sc-17t4b9u-2" }) # grab scores 
    prof_class = soup.findAll("div", {"class": "RatingHeader__StyledClass-sc-1dlkqw1-2 gxDIt" }) # just take the odds
    comment_thumbs = soup.findAll("div", {"class": "VoteThumb__StyledVoteThumb-p2gtch-0" }) # thumbs up and down
    ## split up into difficulty, quality, class num, thumbs up and thumbs down
    prof_dif = prof_score[1::2]
    prof_qual = prof_score[0::2]
    prof_class = prof_class[1::2]
    thumbs_down = comment_thumbs[1::2]
    thumbs_up = comment_thumbs[0::2]
    ### now a new loop to store the scraped data from the jth row into the new temp df 
    d1 = []
    for i, element in enumerate(prof_qual):
        d1.append({
            'row': i+1,
            'quality_of_class': prof_qual[i].get_text(),
            "difficulty_of_class": prof_dif[i].get_text(),
            "thumbs_up": thumbs_up[i].get_text(),
            "thumbs_down": thumbs_down[i].get_text(),
            "class_code": prof_class[i].get_text(),
            #"college": college[j], #jth lvl, not ith level, variable 
            #"prof_firstname": prof_firstname[j],            
            #"prof_lastname": prof_lastname[j], # same logic as above, now for the prof's last name         
            'comment': prof_comments[i].get_text() #ith level again 
        }
        )
    df = pd.DataFrame(d1)
    ## add j level vars here
    df['college'] = college[j]
    df['prof_firstname'] = prof_firstname[j]
    df['prof_lastname'] = prof_lastname[j]
    
    ### now save the file 
    save_name = "scraped_data/thumbs" +"_"+college[j] +"_"+prof_lastname[j]+"_"+prof_firstname[j]+".csv"
    df.to_csv(save_name, index=False)

In [40]:
comment_thumbs

[<div class="Thumbs__HelpTotal-sc-19shlav-1 gbMMhq" id="thumbs_up"><img alt="Thumbs up" class="VoteThumb__StyledVoteThumb-p2gtch-0 feDtbj" data-for="GLOBAL_TOOLTIP" data-tip="Helpful" data-tooltip="true" src="/static/media/thumbs-up-black.eddae738.svg"/> <!-- -->1</div>,
 <div class="Thumbs__HelpTotal-sc-19shlav-1 gbMMhq" id="thumbs_down"><img alt="Thumbs down" class="VoteThumb__StyledVoteThumb-p2gtch-0 feDtbj" data-for="GLOBAL_TOOLTIP" data-tip="Not helpful" data-tooltip="true" src="/static/media/thumbs-down-black.bd601b36.svg"/> <!-- -->0</div>,
 <div class="Thumbs__HelpTotal-sc-19shlav-1 gbMMhq" id="thumbs_up"><img alt="Thumbs up" class="VoteThumb__StyledVoteThumb-p2gtch-0 feDtbj" data-for="GLOBAL_TOOLTIP" data-tip="Helpful" data-tooltip="true" src="/static/media/thumbs-up-black.eddae738.svg"/> <!-- -->1</div>,
 <div class="Thumbs__HelpTotal-sc-19shlav-1 gbMMhq" id="thumbs_down"><img alt="Thumbs down" class="VoteThumb__StyledVoteThumb-p2gtch-0 feDtbj" data-for="GLOBAL_TOOLTIP" data-

In [44]:
### Now let's grab only the OSU files 

#### compile all of the files in the directory ## 
scraped_data_files =  glob.glob("scraped_data/*")
scraped_data_files_osu = [x for x in scraped_data_files if re.findall("FINAL",x)] # grab files that have "zest" in title
scraped_data_files_osu

['scraped_data\\FINALTHUMBS_OHIO STATE UNIVERSITY_ACS_ALEX.csv',
 'scraped_data\\FINALTHUMBS_OHIO STATE UNIVERSITY_ADANIYA_NAOMI.csv',
 'scraped_data\\FINALTHUMBS_OHIO STATE UNIVERSITY_ANDERSON_SARA.csv',
 'scraped_data\\FINALTHUMBS_OHIO STATE UNIVERSITY_ANDRIDGE_REBECCA.csv',
 'scraped_data\\FINALTHUMBS_OHIO STATE UNIVERSITY_ANGEL_WILLIAM.csv',
 'scraped_data\\FINALTHUMBS_OHIO STATE UNIVERSITY_ARCHER_KELLIE.csv',
 'scraped_data\\FINALTHUMBS_OHIO STATE UNIVERSITY_ARKES_HAL.csv',
 'scraped_data\\FINALTHUMBS_OHIO STATE UNIVERSITY_ASHER_HERB.csv',
 'scraped_data\\FINALTHUMBS_OHIO STATE UNIVERSITY_BAKER_JEREMY.csv',
 'scraped_data\\FINALTHUMBS_OHIO STATE UNIVERSITY_BARBOZA-SALERNO_GIA.csv',
 'scraped_data\\FINALTHUMBS_OHIO STATE UNIVERSITY_BASTA_NICHOLAS.csv',
 'scraped_data\\FINALTHUMBS_OHIO STATE UNIVERSITY_BAUM_LAWRENCE.csv',
 'scraped_data\\FINALTHUMBS_OHIO STATE UNIVERSITY_BECK_PAUL.csv',
 'scraped_data\\FINALTHUMBS_OHIO STATE UNIVERSITY_BECK_STEVEN.csv',
 'scraped_data\\FINALTHUMBS_O

In [45]:
### now loop in and read the data 
master_thumbs_osu = pd.DataFrame()

## note: it appears that there are some csvs that simply have no data (i.e. Box Steffensmier.) We will therefore want to go 
#and discard those profs that are characterized by these, and then proceed to skip them, though make note of them. 
## see the use of the try command for this purpose 
  
for element in master_thumbs_osu:
    try:
        temp_df = pd.read_csv(element)
        temp_len = len(temp_df)
        master_osu = master_osu.append(temp_df)
    except: ## passes error, though notes it 
        pass
        print("The evals for ", element, "not appended; had length of", temp_len)

#The evals for  scraped_data\rmp_data_OHIO STATE UNIVERSITY_BOX-STEFFENSMEIER_JANET.csv not appended; had length of 11
#The evals for  scraped_data\rmp_data_OHIO STATE UNIVERSITY_GUNTHER_RICHARD.csv not appended; had length of 15
#The evals for  scraped_data\rmp_data_OHIO STATE UNIVERSITY_HUFTON_AMIE.csv not appended; had length of 20
#The evals for  scraped_data\rmp_data_OHIO STATE UNIVERSITY_IBARAKI_MOTOMU.csv not appended; had length of 19
#The evals for  scraped_data\rmp_data_OHIO STATE UNIVERSITY_KRAJBICH_IAN.csv not appended; had length of 10
#The evals for  scraped_data\rmp_data_OHIO STATE UNIVERSITY_MCGRAW_KATHLEEN.csv not appended; had length of 2
#The evals for  scraped_data\rmp_data_OHIO STATE UNIVERSITY_PATSON_NIKOLE.csv not appended; had length of 20
#The evals for  scraped_data\rmp_data_OHIO STATE UNIVERSITY_REMPALA_GRZEGORZ.csv not appended; had length of 6
    
           
        

In [46]:
### We will want to specially scrape these profs again 

### create a list object 
missing_list = ['GUNTHER','MCGRAW','KRAJBICH']
#missing_list = ['GUNTHER','HUFTON','KRAJBICH','MCGRAW','PATSON','REMPALA']

## MOTOMU IBARAKI is missing viable url, and so is BOX-STEFFENSMEIER ; so is Amie Hufton ; so is Grzegorz Rempala
# so is PATSOn, 
## last two should be good 

## subset the faculty chr list 
faculty_csv_miss = faculty_csv[faculty_csv['prof_lastname'].isin(missing_list)]

#j_range = list(range(0,len(faculty_csv_miss),1))

#faculty_csv_miss.set_index(j_range)
faculty_csv_miss=faculty_csv_miss.reset_index()

#df[df['dog_name'].isin({'Fido', 'Yeller'})]
faculty_csv_miss #looks like it worked 

,index,college,prof_firstname,prof_lastname,program,male,non_white,link_rmp,NOTES
0,103,OHIO STATE UNIVERSITY,IAN,KRAJBICH,PSYCHOLOGY,1,0,https://www.ratemyprofessors.com/professor/227...,NaN
1,221,OHIO STATE UNIVERSITY,RICHARD,GUNTHER,POLITICAL SCIENCE,1,0,https://www.ratemyprofessors.com/professor/175034,NaN
2,224,OHIO STATE UNIVERSITY,KATHLEEN,MCGRAW,POLITICAL SCIENCE,0,99,https://www.ratemyprofessors.com/professor/175678,NaN


In [47]:
### format var list 
urls = faculty_csv_miss['link_rmp']


## url to use ; will be the profs page, with the numeric code being what changes; should be changed manually 

#url = ['https://www.ratemyprofessors.com/professor/1190096','https://www.ratemyprofessors.com/professor/2070585']

## college 
college = faculty_csv_miss['college']

##prof last name 
prof_lastname = faculty_csv_miss['prof_lastname']

# prof first name 
prof_firstname = faculty_csv_miss['prof_firstname']

### need to reindex 


In [53]:
## we now should be good; let's redo 
scraped_data_files =  glob.glob("scraped_data/*")
scraped_data_files_thumbs = [x for x in scraped_data_files if re.findall("FINAL",x)] # grab files that have "zest" in title

## now the append loop 
thumbs_osu = pd.DataFrame()

## note: it appears that there are some csvs that simply have no data (i.e. Box Steffensmier.) We will therefore want to go 
#and discard those profs that are characterized by these, and then proceed to skip them, though make note of them. 
## see the use of the try command for this purpose 
  
for element in scraped_data_files_thumbs:
    try:
        temp_df = pd.read_csv(element)
        temp_len = len(temp_df)
        thumbs_osu = thumbs_osu.append(temp_df)
    except: ## passes error, though notes it 
        pass
        print("The evals for ", element, "not appended; had length of", temp_len)
        
thumbs_osu.to_csv("thumbs2.csv", index=False)

C:\Users\Gabem\AppData\Local\Temp\ipykernel_22124\1529817454.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  thumbs_osu = thumbs_osu.append(temp_df)
C:\Users\Gabem\AppData\Local\Temp\ipykernel_22124\1529817454.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  thumbs_osu = thumbs_osu.append(temp_df)
C:\Users\Gabem\AppData\Local\Temp\ipykernel_22124\1529817454.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  thumbs_osu = thumbs_osu.append(temp_df)
C:\Users\Gabem\AppData\Local\Temp\ipykernel_22124\1529817454.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  thumbs_osu = thumbs_osu.append(temp_df)
C:\Users\Gabem\AppData\Local